# Projet Recherche operationnelle

## **The Robust Spanning Tree Problem With Interval Data**

### Yahya Chammami

**A mixed integer programming formulation**

\begin{align*}
\text{minimize} \quad & \sum_{e \in E} c_e \cdot x_e \\
\text{subject to} \quad & \sum_{(i, j) \in A} f_{ij} - \sum_{(j, i) \in A} f_{ji} = \begin{cases}
n - 1 & \text{if } i = 1 \\
-1 & \forall i \in V \setminus \{1\} \\
\end{cases} \\
& f_{ij} \leq (n - 1) x_{ij} \quad \forall \{i, j\} \in E \\
& f_{ji} \leq (n - 1) x_{ij} \quad \forall \{i, j\} \in E \\
& \sum_{e \in E} x_e = n - 1 \\
& f_{ij} \geq 0 \\
& x_e \in \{0, 1\} \quad \forall e \in E
\end{align*}

# Introduction
La conception de réseaux robustes dans le domaine des télécommunications et des technologies de l'information est cruciale pour assurer des communications fiables et efficaces.

 Un défi majeur dans cette conception réside dans la gestion des incertitudes liées aux coûts des liaisons entre les nœuds du réseau. Ces incertitudes peuvent découler de facteurs tels que la variation de la charge de trafic, les fluctuations dans les retards de transmission, ou d'autres sources d'instabilité.

Le présent travail explore une approche novatrice pour résoudre le problème de conception d'un réseau robuste, en utilisant la programmation linéaire pour formuler et résoudre le "Robust Spanning Tree Problem with Interval Data." Ce problème cherche à déterminer une structure de réseau robuste, représentée par un arbre de recouvrement, qui minimise les coûts tout en prenant en compte l'incertitude inhérente aux coûts des liaisons.

### Contexte de Résolution :

Le "Robust Spanning Tree Problem with Interval Data" s'inscrit dans le cadre de deux applications majeures dans l'industrie des télécommunications. Tout d'abord, dans la conception de réseaux de communication, où les retards de routage sur les liaisons ne sont pas connus avec certitude en raison de la nature variable de la charge de trafic. Dans de telles situations, il est essentiel de développer une configuration de réseau qui se prémunit contre les contingences les plus défavorables en termes de retards de routage.

En utilisant cette approche, le travail contribue à l'avancement des techniques de conception de réseaux robustes, offrant une méthodologie systématique pour résoudre le problème de l'arbre de recouvrement robuste dans des conditions d'incertitude réalistes. Les résultats obtenus grâce à la résolution du système fournissent des informations cruciales pour la conception de réseaux résilients dans des environnements dynamiques et imprévisibles.


#### **importation des libraries**

In [16]:
import numpy as np
from scipy.optimize import linprog

**Fonction Objectif**

Minimiser la somme des coûts pondérés des arêtes sélectionnées. La fonction objectif vise à trouver la solution qui optimise le coût total de la conception du réseau.

\begin{align*}
\text{Min}\quad & \sum_{e \in E} c_e \cdot x_e
\
\end{align*}

In [17]:
# Nombre de nœuds
n = 5

# Générer des coûts aléatoires pour chaque arête
c_edges = np.random.rand(n, n)

# Générer une matrice d'incidence orientée aléatoire
A = np.random.randint(0, 2, size=(n, n))

# Définir la fonction objectif pour minimiser la somme des coûts
c = c_edges.flatten()

A

array([[1, 0, 1, 0, 1],
       [1, 0, 1, 1, 0],
       [1, 1, 0, 1, 0],
       [1, 1, 0, 1, 1],
       [1, 1, 0, 0, 1]])

**Conservation du Flux** :


Cette contrainte garantit que, pour chaque nœud du réseau, la quantité totale de flux entrant est égale à la quantité totale de flux sortant. Cela assure l'équilibre du flux à chaque nœud, préservant ainsi la cohérence du réseau.


\begin{align*}
\\
\quad & \sum_{(i, j) \in A} f_{ij} - \sum_{(j, i) \in A} f_{ji} = \begin{cases}
n - 1 & \text{if } i = 1 \\
-1 & \forall i \in V \setminus \{1\}
\end{cases}
\end{align*}



In [18]:
# Définir les contraintes d'équilibre du flux
A_eq = []
b_eq = []

for i in range(1, n):
    row = np.zeros(n**2)
    row[(i-1)*n:i*n] = 1  # Flux sortant de i
    row[i-1::n] = -1     # Flux entrant à i
    A_eq.append(row)
    b_eq.append(-1 if i != 1 else n - 1)

A_eq = np.array(A_eq)
b_eq = np.array(b_eq)


**Capacité de Flux** :

 Cette contrainte définit la capacité maximale de chaque arête en fonction de sa sélection. Si une arête est sélectionnée (Xij=1)   le flux sur cette arête (Fij) ne peut pas dépasser une certaine limite. Cela garantit que les flux sont conformes aux capacités des arêtes.

 \begin{align*}
\
& f_{ij} \leq (n - 1) x_{ij} \quad \forall \{i, j\} \in E \\
& f_{ji} \leq (n - 1) x_{ij} \quad \forall \{i, j\} \in E \\
\
\end{align*}

In [19]:
# Définir les contraintes de capacité
A_leq = []
b_leq = []

for i in range(n):
    for j in range(n):
        if A[i, j] == 1:
            row = np.zeros(n**2)
            row[i*n + j] = -(n - 1)
            A_leq.append(row)
            b_leq.append(0)

A_leq = np.array(A_leq)
b_leq = np.array(b_leq)

**Cardinalité** :

 Il s'agit d'une contrainte de cardinalité qui stipule que le nombre total d'arêtes sélectionnées doit être égal à (n-1) assurant ainsi la formation d'un arbre de recouvrement.

\begin{align*}
\sum_{e \in E} x_e = n - 1 \\
\
\end{align*}

In [20]:
# Définir la contrainte du nombre d'arêtes sélectionnées
A_eq_edges = np.ones((1, n**2))
b_eq_edges = n - 1

**Non-négativité des Flux**  :

Les flux sur chaque arête (Fij) doivent être non négatifs, ce qui est une condition réaliste pour les quantités de flux physiques.

\begin{align*}
\
& f_{ij} \geq 0 \\
\
\end{align*}

**Binarité des Variables de Sélection**  :

Les variables de sélection des arêtes Xe sont binaires, ce qui signifie qu'une arête est soit sélectionnée (Xe=1) soit non sélectionnée (Xe=0) Cela reflète le caractère discret de la décision d'inclure ou non une arête dans l'arbre de recouvrement.

\begin{align*}
\
& x_e \in \{0, 1\} \quad \forall e \in E
\end{align*}




In [21]:
# Définir Binarité pour les variables
bounds = [(0, None) for _ in range(n**2)]


**Resolution de systeme**

In [22]:
# Résoudre le problème d'optimisation linéaire
result = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_leq, b_ub=b_leq, bounds=bounds, method='highs')

# Afficher les résultats
print("Solution optimale:")
print(result.x.reshape((n, n)))
print("Coût optimal:", result.fun)


Solution optimale:
[[0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Coût optimal: 2.209914655662711


## Conclusion :

La solution optimale montre qu'un arbre couvrant optimal a été trouvé. Seules les arêtes (1, 2), (1, 3), (1, 4), et (1, 5) sont incluses dans cet arbre. Les autres arêtes ont une valeur de "0", indiquant qu'elles ne font pas partie de l'arbre couvrant optimal.

Le coût optimal associé à cette solution est d'environ 2.209. Cela représente la somme des coûts des arêtes incluses dans l'arbre couvrant optimal.

Cette solution offre des perspectives pratiques pour la conception de réseaux robustes dans des environnements sujets à des variations de coûts et de délais. La sélection stratégique des arêtes dans des conditions d'incertitude contribue à la création de réseaux résilients, capables de faire face aux pires scénarios de contingence.

En conclusion, ce projet apporte une contribution significative à la conception de réseaux robustes et ouvre des pistes pour des applications pratiques dans le domaine des télécommunications et au-delà. Les résultats obtenus fournissent des bases solides pour la prise de décisions éclairées dans des environnements dynamiques et incertains.